In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, preprocessing, models, layers, optimizers, losses
import matplotlib.pyplot as plt
import numpy as np
import time

In [9]:
# Download Shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# Read and decode text
text = open(path_to_file, 'rb').read().decode(encoding='utf8')
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

